In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from sklearn.neighbors import LocalOutlierFactor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.metrics import accuracy_score

import tensorflow as ts
from tensorflow import keras

In [ ]:
#df1=pd.read_csv('/kaggle/input/heart-attack-analysis-prediction-dataset/o2Saturation.csv')
data=pd.read_csv('/kaggle/input/heart-attack-analysis-prediction-dataset/heart.csv')

In [ ]:
dataV = data.copy()
dataV["sex"] = pd.Categorical(dataV["sex"])
dataV["cp"] = pd.Categorical(dataV["cp"])
dataV["fbs"] = pd.Categorical(dataV["fbs"])
dataV["restecg"] = pd.Categorical(dataV["restecg"])
dataV["exng"] = pd.Categorical(dataV["exng"])
dataV["slp"] = pd.Categorical(dataV["slp"])
dataV["caa"] = pd.Categorical(dataV["caa"])
dataV["thall"] = pd.Categorical(dataV["thall"])
dataV["output"] = pd.Categorical(dataV["output"])

df = data.select_dtypes(include=["float64","int64","int32"])

# Data Analysis

In [ ]:
df.describe()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

df.hist(figsize=(20,10))
plt.show()

# Pearson Correlation

In [ ]:
correlation_mat = df.corr()
figure = plt.figure(figsize=(10,8))
sns.heatmap(correlation_mat,annot=True,cmap='RdYlGn', vmin=-1, vmax=+1)
plt.title("PEARSON")
plt.xlabel("COLUMNS")
plt.ylabel("COLUMNS")
plt.show()

# Outlier

In [ ]:

#Explanation: -1 is outlier, +1 is inliner
out=data.copy()
clf = LocalOutlierFactor(n_neighbors=2)
clf.fit_predict(out)

In [ ]:
score=clf.negative_outlier_factor_
scoresorted=np.sort(score)
print(scoresorted[0:30])

In [ ]:
#Example for one outlier
point = scoresorted[12]
print(out[score == point])

# Models

In [ ]:
x = data.drop("output",axis=1)
y = data["output"]

xTrain,xTest,yTrain,yTest = train_test_split(x,y,test_size=0.20,random_state=42)

scaler = MinMaxScaler().fit(xTrain,yTrain)
xRTrain = scaler.transform(xTrain)
xRTest = scaler.transform(xTest)

# Classification

In [ ]:
names = ["Nearest Neighbors", "Linear SVM", "RBF SVM", "Gaussian Process",
         "Decision Tree", "Random Forest", "Neural Net", "AdaBoost",
         "Naive Bayes", "QDA"]
classifiers = [
    KNeighborsClassifier(3),
    SVC(kernel="linear", C=0.025),
    SVC(gamma=2, C=1),
    GaussianProcessClassifier(1.0 * RBF(1.0)),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    MLPClassifier(alpha=1, max_iter=1000),
    AdaBoostClassifier(),
    GaussianNB(),
    QuadraticDiscriminantAnalysis()]

scores=np.zeros(len(classifiers))

for name, clf,i in zip(names, classifiers,range(len(scores))):
        clf.fit(xRTrain, yTrain)
        score = clf.score(xRTest, yTest)
        scores[i]=score
        #print(name,np.round_(score,3))

figure = plt.figure(figsize=(10,8))
sns.barplot(x=scores,y=names)
plt.title('Classification')
plt.xlim=(0,1)
plt.show()

# Neuronal networks

In [ ]:
model = keras.Sequential([
    keras.layers.Dense(64, activation='relu',input_dim=xRTrain.shape[1]),
    keras.layers.Dense(16, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid'),
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(xRTrain,yTrain,epochs=200,batch_size = 32,validation_split = 0.2);

In [ ]:
yPredict=(model.predict(xRTest) > 0.5).astype("int32")
NNscore = accuracy_score(yTest, yPredict)
print('NN has a accuracy score of: ',np.round_(NNscore,3))